In [1]:
import pandas as pd
import requests
from datetime import datetime, timedelta
import time
import os

def fetch_data_range(crypto, start_date, end_date):
    # Create an empty DataFrame to hold all data
    all_data = pd.DataFrame()

    current_date = start_date
    while current_date < end_date:
        next_date = current_date + timedelta(days=1)

        # API endpoint and parameters
        url = f"https://api.coingecko.com/api/v3/coins/{crypto}/market_chart/range"
        params = {
            'vs_currency': 'usd',
            'from': int(current_date.timestamp()),
            'to': int(next_date.timestamp())
        }

        # Fetch data using the safe_request function
        response = safe_request(url, params)

        if response and response.status_code == 200:
            data = response.json()
            df = pd.DataFrame(data['prices'], columns=['date', 'price'])
            df['date'] = pd.to_datetime(df['date'], unit='ms')
            df['market_cap'] = pd.DataFrame(data['market_caps'])[1].values
            df['volume_24h'] = pd.DataFrame(data['total_volumes'])[1].values

            # Append the daily data to the all_data DataFrame
            all_data = pd.concat([all_data, df], ignore_index=True)
        else:
            print(f"Failed to fetch data for {current_date}. Status code: {response.status_code}" if response else "Failed to fetch data; no response.")

        # Move to the next day
        current_date = next_date

    # Save the combined DataFrame to a CSV file
    os.makedirs('../data/raw/minute5/', exist_ok=True)
    all_data.to_csv('../data/raw/minute5/ethereum.csv', index=False)
    return all_data

# Helper function for handling API requests with retries
def safe_request(url, params, retries=5, backoff_factor=0.5):
    for i in range(retries):
        try:
            response = requests.get(url, params=params)
            if response.status_code == 200:
                return response
            elif response.status_code == 429:
                # Handle rate limiting
                sleep_time = backoff_factor * (2 ** i)
                print(f"Rate limit hit. Waiting {sleep_time:.2f} seconds before retrying...")
                time.sleep(sleep_time)
            else:
                # Other errors, log and break the retry loop
                print(f"Request failed with status code {response.status_code}.")
                return None
        except requests.exceptions.RequestException as e:
            print(f"Request exception: {e}. Retrying...")
            time.sleep(backoff_factor * (2 ** i))
    return None


In [5]:
# Usage
start_date = datetime(2023, 1, 1)
end_date = datetime(2023, 1, 2)
fetch_data_range('ethereum', start_date, end_date)

Request failed with status code 401.
Failed to fetch data; no response.


""
